# <center>Supervised Learning - Text Classification</center>
References:
* http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

## 1. What we'll cover in this lecture
* Review basic concepts of machine learning
  * Cross validation
  * Performance metrics: recall and precision
* Text Classification  
  * Assign a document into one  or more pre-defined categories (or labels)
    * Input: 
      - a document $d$ 
      - a fixed set of classes C = {$c_1$, $c_2$,..., $c_J$}
      - A training set of $m$ hand-labeled documents ($d_1,c_1$),....,($d_m,c_m$)
    * Output: a classifier that predicts $d$ to some classes $c$ $\subset$ C
  * **Single-label** classification: e.g. spam dection
  * **Multi-label** classification: e.g. news categorization

## 2. Review basic concepts of machine learning
### 2.1. Cross validation 
  * Defition: model valiation technique for assessing how a model will generalize to an independent data set
  * *k*-fold cross validation: 
    - Often, there is not enough data to allow some of it to be kept back for testing. 
    - Thus, the data is separated into k subsets. Each time, one of the subsets is held as the test set (a.k.a holdout) and the rest of them is used as the training set. 
    - This method repeats *k* times and each time with a different subset as the test set. 
  <img src="cross_validation.png" width="50%"> [source] (http://spark-public.s3.amazonaws.com/nlp/slides/sentiment.pptx)

### 2.2. Performance metrics
  * Precision: precentage of true cases among the predicated true cases
  * Recall:  precentage of true cases that have been retrieved over the total number of true cases
  * F-score: $$\frac{2*precision*recall}{precision+recall}$$
  * Example: 
Confusion Matrix: <img src="confusion_matrix.png">
    * For "YES" group: 
      - precision=100/(100+10)=0.91, 
      - recall=100/(100+5)=0.95, 
      - f-score=2\*0.91\*0.95/(0.91+0.95)=0.92
      <img src="precision_recall.png" width="60%">
    * For "NO" group:
      - precision=?, 
      - recall=?, 
      - f-score=?
  * Overall model performance
    * precision_macro (or recall_macro or f1_macro) is calculated as:
      1. calculate precision for each label
      2. average over labels 
    * pericision_micro (or recall_micro or f1_micro): calculates metrics globally regardless of labels
    * With inbalanced classes, the difference between these two metrics may be significant

## 3. Text Classification

* Basic process
  1. Load and preprocess training data
  2. Extract features: e.g. bag of words with TF-IDF weights
  3. Split data into trainning and test sets following cross validation method
  4. Train a classifier/model with the training dataset using selected classification algorithm for each fold
  5. Calculate performance
 
* Considerations for deciding text classification algorithms
  - should be effective in high dimensional spaces
  - should be effective even if the number of features is greater than the number of samples
    * Is regression a good alogorithm if you have a small number of text samples?
  - some good algorithms to start with:
      - Naive Bayes (https://web.stanford.edu/class/cs124/lec/naivebayes.pdf): baseline for performance benchmarking of text classification algorithms
      - Support Vector Machine (SVM) (https://www.svm-tutorial.com/2014/11/svm-understanding-math-part-1/)
  

In [2]:
# Exercise 3.1.: Load data 
# Load datasets (http://qwone.com/~jason/20Newsgroups/)
# For convenience, a subset of the data has been saved into "twenty_news_data.pkl"

import pickle
data=pickle.load(open("twenty_news_data.pkl","r"))

# data is a list of tuple (text, label)
# print out the first document
print("Text: "+data[0][0]+"\nGroup: "+data[0][1])

Text: From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.

Group: comp.graphics


In [3]:
# Exercise 3.2. Prepare text and class labels

# split data as list of tuples into text and target tuples
text,target=zip(*data)

# convert tuple to list
text=list(text)
target=list(target)


In [12]:
type(text)

list

## 3.1. TF-IDF matrix generation
- Function: **sklearn.feature_extraction.text.TfidfVectorizer**(input='content',encoding='utf-8', decode_error='strict', token_pattern='(?u)\b\w\w+\b', lowercase=True, stop_words=None, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, norm='l2', use_idf=True, smooth_idf=True, ...)
- Some useful parameters:
    * **input** : string {'filename', 'file', 'content').
    * **encoding** : string, 'utf-8' by default.
If bytes or files are given to analyze, this encoding is used to decode.
    * **decode_error** : {'strict', 'ignore', 'replace'}: Instruction on what to do if a byte sequence is given to analyze that contains characters not of the given encoding. By default, it is 'strict', meaning that a UnicodeDecodeError will be raised. Other values are ‘ignore’ and ‘replace’.
    * **token_pattern** : Regular expression denoting what constitutes a “token”. The default is '(?u)\b\w\w+\b', i.e. a token contains at least two word characters in unicode (note: ?u: unicode, \b: space or non-word character, i.e. boundary, \w: word character). 
    * **ngram_range** : tuple (min_n, max_n): The lower and upper boundary of the range of n-values for different n-grams to be extracted. 
    * **stop_words** : string {‘english’}, list, or None (default)
    * **lowercase** : boolean, default True: Convert all characters to lowercase before tokenizing.
    * **max_df/min_df** : float in range [0.0, 1.0] or int, default=1.0: When building the vocabulary ignore terms that have a document frequency strictly higher (lower) than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. 
    * **max_features** : int or None, default=None. If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
    * **norm** : 'l1', 'l2' or None, optional. Norm used to normalize term vectors. None for no normalization.
    * **use_idf** : boolean, default=True. Enable inverse-document-frequency reweighting.
    * **smooth_idf** : boolean, default=True. Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.
- For all the parameters, see http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


In [2]:
# Exercise 3.3. Create TF-IDF Matrix

from sklearn.feature_extraction.text import TfidfVectorizer

# initialize the TfidfVectorizer 

tfidf_vect = TfidfVectorizer() 

# with stop words removed
#tfidf_vect = TfidfVectorizer(stop_words="english") 

# generate tfidf matrix
dtm= tfidf_vect.fit_transform(text)

print("type of dtm:", type(dtm))
print("size of tfidf matrix:", dtm.shape)

NameError: name 'text' is not defined

#for scripy sparse matrix, use data to read non zero values in the matrix; use indices to read corresponding index.

In [5]:
dtm[0].data

array([ 0.01679781,  0.13487106,  0.31440007,  0.12491818,  0.11819702,
        0.19622799,  0.38418039,  0.01679781,  0.21567206,  0.0744441 ,
        0.07283774,  0.17358472,  0.24645541,  0.18626015,  0.03637492,
        0.03429071,  0.03604415,  0.11382739,  0.01776232,  0.08865416,
        0.06567578,  0.01686489,  0.05966162,  0.03779319,  0.043162  ,
        0.03478259,  0.01824994,  0.04270369,  0.04334165,  0.06866113,
        0.07802072,  0.08413454,  0.0979625 ,  0.099941  ,  0.07830787,
        0.12806013,  0.1232277 ,  0.10218403,  0.13635772,  0.04525255,
        0.07691883,  0.03448147,  0.03127031,  0.03900412,  0.03590436,
        0.03104295,  0.04727158,  0.1232277 ,  0.11947938,  0.04935883,
        0.1256015 ,  0.03109515,  0.06899051,  0.01996488,  0.02387114,
        0.06350566,  0.05417404,  0.04910237,  0.01894546,  0.06866113,
        0.08497461,  0.04967185,  0.09313008,  0.08631915,  0.25612026,
        0.24645541,  0.10783603,  0.13487106,  0.10960586,  0.06

In [6]:
# Exercise 3.4. Examine TF-IDF

# 1. Check vocabulary
# Vocabulary is a dictionary mapping a word to an index
print("type of vocabulary:", type(tfidf_vect.vocabulary_))
print("index of word 'city' in vocabulary:", tfidf_vect.vocabulary_['city'])

# the number of words in the vocabulary
print("total number of words:", len(tfidf_vect.vocabulary_ ))

# 2. check the weight of words in a document, e.g. 1st document

# first, covert the sparse matrix row to a list
doc0=dtm[0].toarray()[0]

print("\nOriginal text: \n"+text[0])
print("\ntfidf weights: \n")

# second, get mapping from word index to word
# i.e. reversal mapping of tfidf_vect.vocabulary_
voc_lookup={tfidf_vect.vocabulary_[word]:word \
            for word in tfidf_vect.vocabulary_}

# get words with tfidf weight>0
tfidf_words=[(voc_lookup[i], doc0[i]) for i in range(len(doc0)) if doc0[i]>0]
print(sorted(tfidf_words, key=lambda item:-item[1]))

('type of vocabulary:', <type 'dict'>)
("index of word 'city' in vocabulary:", 8696)
('total number of words:', 35788)

Original text: 
From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.


tfidf weights: 

[(u'collier', 0.3841803935867984), (u'city', 0.31440006552897398), (u'071', 0.25612026239119895), (u'477', 0.24645540709354397), (u'laserjet', 0.24645540709354

multinomial Naive bayes;
for a text datasets, D documents, V vocabulary including x_i, i in {0, ... n} words, for each d, word x from V.
C classes of topics.
P(c_j) = count of c_j / total count of documents
P(x_i | c_j) = count of x_i in c_j / total words in c_j
P(x_i) = count of x_i in all documents / total words

P(c_j | x_d_k) approx. => P(x_d_k | c_j) * P(c_j), where x_d_k is set of words in d_k
Using naive Bayes rule, assuming each word is independent of each other.
P(x_d_k | c_j) = P(x_0_d_k | c_j) * P(x_1_d_k | c_j) * ... * P(x_m_d_k | c_j)

In [7]:
# Exercise 3.5. classification using a single fold

# use MultinomialNB algorithm
from sklearn.naive_bayes import MultinomialNB

# import method for split train/test data set
from sklearn.model_selection import train_test_split

# import method to calculate metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn import metrics

# split dataset into train (70%) and test sets (30%)
X_train, X_test, y_train, y_test = train_test_split(\
                dtm, target, test_size=0.3, random_state=0)

# train a multinomial naive Bayes model using the testing data
clf = MultinomialNB(alpha = 0).fit(X_train, y_train)

# predict the news group for the test dataset
predicted=clf.predict(X_test)

# get the list of unique labels
labels=list(set(target))

# calculate performance metrics. 
# Support is the number of occurrences of each label

precision, recall, fscore, support=\
     precision_recall_fscore_support(y_test, predicted, labels=labels)

print(labels)
print(precision)
print(recall)
print(fscore)
print(support)

# another way to get all performance metrics
print(metrics.classification_report(y_test, predicted, target_names=labels))

['sci.med', 'soc.religion.christian', 'alt.atheism', 'comp.graphics']
[ 0.91758242  0.95675676  0.97222222  0.96407186]
[ 0.97093023  0.94148936  0.95890411  0.93604651]
[ 0.94350282  0.94906166  0.96551724  0.94985251]
[172 188 146 172]
                        precision    recall  f1-score   support

               sci.med       0.97      0.96      0.97       146
soc.religion.christian       0.96      0.94      0.95       172
           alt.atheism       0.92      0.97      0.94       172
         comp.graphics       0.96      0.94      0.95       188

           avg / total       0.95      0.95      0.95       678



E:\ProgramData\anaconda2\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


When alpha = 0, performace improved significantly.

In [8]:
# Exercise 3.6.  predict new documents

docs_new = ['God is love', 'OpenGL on the GPU is fast']

# generate tifid for new documents
X_new_tfidf = tfidf_vect.transform(docs_new)

print(X_new_tfidf.shape)

# predict classes for new documents
predicted = clf.predict(X_new_tfidf)

for idx, doc in enumerate(docs_new):
    print('%r => %s' % (doc, predicted[idx]))


(2, 35788)
'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [ ]:
# Exercise 3.7. Classification with stop words removed
# Can removing stop words improves performance?
# In Exercise 3.3, uncomment line 10 and comment line 7
# Run Exercise 3.3
# Run Exercise 3.5

In [9]:
# Exercise 3.8. Run 5 fold cross validation
# to show the generalizability of the model

# import cross validation method
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import MultinomialNB

metrics = ['precision_macro', 'recall_macro', "f1_macro"]

#clf = MultinomialNB()
clf = MultinomialNB(alpha=0.1)

cv = cross_validate(clf, dtm, target, scoring=metrics, cv=5)
print("Test data set average precision:")
print(cv['test_precision_macro'])
print("\nTest data set average recall:")
print(cv['test_recall_macro'])
print("\nTest data set average fscore:")
print(cv['test_f1_macro'])

# To see the performance of training data set use cv['train_xx_macro']

# The metrics are quite stable across folds.
# The performance between training and test sets is small
# This indicates the model has good generality

Test data set average precision:
[ 0.97733303  0.97465317  0.96643912  0.97238159  0.97849985]

Test data set average recall:
[ 0.97435482  0.97221897  0.96284397  0.97017065  0.9774104 ]

Test data set average fscore:
[ 0.97561608  0.97320334  0.96424212  0.97114142  0.97781257]


In [43]:
cv

{'fit_time': array([ 0.04247713,  0.01781797,  0.01361394,  0.01305008,  0.01426411]),
 'score_time': array([ 0.01673484,  0.011163  ,  0.006778  ,  0.00617504,  0.00602102]),
 'test_f1_macro': array([ 0.97561608,  0.97320334,  0.96424212,  0.97114142,  0.97781257]),
 'test_precision_macro': array([ 0.97733303,  0.97465317,  0.96643912,  0.97238159,  0.97849985]),
 'test_recall_macro': array([ 0.97435482,  0.97221897,  0.96284397,  0.97017065,  0.9774104 ]),
 'train_f1_macro': array([ 0.99777202,  0.99778033,  0.99946919,  0.99777202,  0.99777532]),
 'train_precision_macro': array([ 0.9979098 ,  0.99792961,  0.99946581,  0.9979098 ,  0.99791416]),
 'train_recall_macro': array([ 0.99764529,  0.99764529,  0.99947368,  0.99764529,  0.9976475 ])}

In [ ]:
# Exercise 3.9. Multinominal NB with different smoothing parameter alpha
# comment line 11 and uncomment 12 in Exercise 3.8
# use different alpha value to see if it affects performance

In [10]:
# Exercise 3.10. SVM model

from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support
from sklearn import svm

metrics = ['precision_macro', 'recall_macro', "f1_macro"]

# initiate an linear SVM model
clf = svm.LinearSVC()

cv = cross_validate(clf, dtm, target, scoring=metrics, cv=5)
print("Test data set average precision:")
print(cv['test_precision_macro'])
print("\nTest data set average recall:")
print(cv['test_recall_macro'])
print("\nTest data set average fscore:")
print(cv['test_f1_macro'])


Test data set average precision:
[ 0.97019783  0.97449526  0.96449132  0.96739007  0.97877082]

Test data set average recall:
[ 0.9680339   0.97376396  0.96232313  0.96444238  0.97789593]

Test data set average fscore:
[ 0.96898048  0.97398282  0.96303024  0.9652361   0.978057  ]


## 3.3. Parameter tuning using grid search
* Each classification model has a few parameters
  * e.g. "stop_words": "english" or None, min_df: [1,2,3, ...]
  * e.g. MultinomialNB(alpha=1.0)
  * e.g. LinearSVC(penalty=’l2’, loss=’squared_hinge’,...)
* Instead of tweaking the parameters of the various components, it is possible to run an exhaustive search of the best parameters on a grid of possible values

In [11]:
# Exercise 3.3.1 Grid search

# import pipeline class
from sklearn.pipeline import Pipeline

# import GridSearch
from sklearn.model_selection import GridSearchCV

# build a pipeline which does two steps all together:
# (1) generate tfidf, and (2) train classifier
# each step is named, i.e. "tfidf", "clf"

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB())
                   ])

# set the range of parameters to be tuned
# each parameter is defined as 
# <step name>__<parameter name in step>
# e.g. mid_df is a parameter of TfidfVectorizer()
# "tfidf" is the name for TfidfVectorizer()
# therefore, 'tfidf__min_df' is the parameter in grid search

parameters = {'tfidf__min_df':[2,3],
              'tfidf__stop_words':[None,"english"],
              'clf__alpha': [0.5,1.0,2.0],
}

# the metric used to select the best parameters
metric =  "f1_macro"

# GridSearch also uses cross validation
gs_clf = GridSearchCV(text_clf, param_grid=parameters, scoring=metric, cv=5)

# due to data volume and large parameter combinations
# it may take long time to search for optimal parameter combination
# you can use a subset of data to test
gs_clf = gs_clf.fit(text, target)


KeyboardInterrupt: 

In [21]:
# gs_clf.best_params_ returns a dictionary 
# with parameter and its best value as an entry

for param_name in gs_clf.best_params_:
    print(param_name,": ",gs_clf.best_params_[param_name])

print("best f1 score:", gs_clf.best_score_)

('tfidf__stop_words', ': ', 'english')
('tfidf__min_df', ': ', 2)
('clf__alpha', ': ', 0.5)
('best f1 score:', 0.96846636442327894)


In [ ]:
# Exercise 3.3.2 Grid search
# Modify Exercise 3.3 and Exercise 3.8 
# to use the best parameters found
# re-create the Multinominal NB classifier


## 4. Multi-label classification
- So far we only cover single-label classification, i.e. assign one class to each sample
- Multilabel classification emerges as a challenging problem, where classes are not mutually exclusive 
  * music categorization 
  * semantic classification of images
  * tagging
- **One-Vs-the-Rest** Strategy (a.k.a **one-vs-all**)
  * fitting one classifier per class. For each classifier, the class is fitted against all the other classes.
  * for $n$ classes (labels), $n$ classifier is needed
  * Advantage: good interpretability - Since each class is represented by one and only one classifier, it is possible to gain knowledge about the class by inspecting its corresponding classifier
  * Disadvantage: 
     * many classifiers are created if there is a large number classes
     * ignore the structure (or dependencies) of classes

In [3]:
# Exercise 4.1 Multi-label classification

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer

# import OneVsRestClassifier, a wrap for all individual classifiers
from sklearn.multiclass import OneVsRestClassifier

X_train = np.array(["new york is a hell of a town",
                    "new york was originally dutch",
                    "the big apple is great",
                    "new york is also called the big apple",
                    "nyc is nice",
                    "people abbreviate new york city as nyc",
                    "the capital of great britain is london",
                    "london is in the uk",
                    "london is in england",
                    "london is in great britain",
                    "it rains a lot in london",
                    "london hosts the british museum",
                    "new york is great and so is london",
                    "i like london better than new york"])

target_names = ['New York', 'London']

# The target needs to be an indicator matrix y_train
# where y_train[i,j] means the jth label of sample i
# e.g. y_train[0,0]=[1,0] means the first sample is 
# labeled with 'New York' but not 'London'

y_train = np.array([[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],\
           [0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[1,1],[1,1]])
X_test = np.array(['nice day in nyc',
                   'welcome to london',
                   'hello welcome to new york. enjoy it here and london too'])   

classifier = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)

print(predicted)

for idx, p in enumerate(predicted):
    predicted_labels=[target_names[x] for x in [0,1] if p[x]==1]
    print('%s => %s' % (X_test[idx], ', '.join(predicted_labels) ))


[[1 0]
 [0 1]
 [1 1]]
nice day in nyc => New York
welcome to london => London
hello welcome to new york. enjoy it here and london too => New York, London
